## Training object detection model

Reference GitHub: https://medium.com/fullstackai/how-to-train-an-object-detector-with-your-own-coco-dataset-in-pytorch-319e7090da5

In [1]:
import os
import torch
import torchvision

from utils import get_model_object_detection, myOwnDataset

In [2]:
root = os.getcwd() + '/../data/birds/annotations/all'
pathin = root + '/../coco_annotations.json'

def get_transform():
    custom_transforms = []
    custom_transforms.append(torchvision.transforms.ToTensor())
    return torchvision.transforms.Compose(custom_transforms)

ds = myOwnDataset(root, pathin, get_transform())

# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))

data_loader = torch.utils.data.DataLoader(ds,
            batch_size = 2,
            shuffle = True,
            num_workers = 0,
            collate_fn = collate_fn)

loading annotations into memory...
Done (t=0.27s)
creating index...
index created!


In [ ]:
# 2 classes; Only target class or background
num_classes = 2
num_epochs = 10
model = get_model_object_detection(num_classes)

# select device (whether GPU or CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# move model to the right device
model.to(device)
    
# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

len_dataloader = len(data_loader)

for epoch in range(num_epochs):
    model.train()
    i = 0    
    for imgs, annotations in data_loader:
        i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        print(f'Iteration: {i}/{len_dataloader}, Loss: {losses}')